<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/master/benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip -q install transformers
!pip -q install sentencepiece
!pip -q install nereval

     |████████████████████████████████| 1.8MB 8.9MB/s 
     |████████████████████████████████| 890kB 32.0MB/s 
     |████████████████████████████████| 2.9MB 44.7MB/s 
     |████████████████████████████████| 1.2MB 9.6MB/s 


In [2]:
import pandas as pd
import numpy as np
from transformers import (
    pipeline, 
    AutoConfig, 
    AutoTokenizer, 
    AutoModel, 
    AutoModelForTokenClassification
)
from pprint import pprint

In [5]:
! rm -rf data
! wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/test.txt
! mkdir data && mv test.txt data/

test.txt            100%[===================>] 108.10K  --.-KB/s    in 0.02s   


# Convert to Text

In [6]:
from pathlib import Path
import re

def convert_lines_to_text(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

In [7]:
texts, tags = convert_lines_to_text('data/test.txt')

In [8]:
s = ''
for word in texts[0]:
  s += word + ' '
s

'خرداد 1399 « کیهان » میراث سناتور مصطفی مصباح\u200cزاده 78 ساله شد کیهان لندن تنها رسانه ایرانی در تبعید است که « از جنگ دوم جهانی تاکنون قدمت و یک موسسه غصب شده در تهران » دارد '

# Benchmark

In [9]:
import logging
from collections import namedtuple
from copy import deepcopy

logging.basicConfig(
    format="%(asctime)s %(name)s %(levelname)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level="DEBUG",
)

Entity = namedtuple("Entity", "e_type start_offset end_offset")

class Evaluator():

    def __init__(self, true, pred, tags):
        """
        """

        if len(true) != len(pred):
            raise ValueError("Number of predicted documents does not equal true")

        self.true = true
        self.pred = pred
        self.tags = tags

        # Setup dict into which metrics will be stored.

        self.metrics_results = {
            'correct': 0,
            'incorrect': 0,
            'partial': 0,
            'missed': 0,
            'spurious': 0,
            'possible': 0,
            'actual': 0,
            'precision': 0,
            'recall': 0,
        }

        # Copy results dict to cover the four schemes.

        self.results = {
            'strict': deepcopy(self.metrics_results),
            'ent_type': deepcopy(self.metrics_results),
            'partial':deepcopy(self.metrics_results),
            'exact':deepcopy(self.metrics_results),
            }

        # Create an accumulator to store results

        self.evaluation_agg_entities_type = {e: deepcopy(self.results) for e in tags}


    def evaluate(self):

        logging.info(
            "Imported %s predictions for %s true examples",
            len(self.pred), len(self.true)
        )

        for true_ents, pred_ents in zip(self.true, self.pred):

            # Check that the length of the true and predicted examples are the
            # same. This must be checked here, because another error may not
            # be thrown if the lengths do not match.

            if len(true_ents) != len(pred_ents):
                raise ValueError("Prediction length does not match true example length")

            # Compute results for one message

            tmp_results, tmp_agg_results = compute_metrics(
                collect_named_entities(true_ents),
                collect_named_entities(pred_ents),
                self.tags
            )

            # Cycle through each result and accumulate

            # TODO: Combine these loops below:

            for eval_schema in self.results:

                for metric in self.results[eval_schema]:

                    self.results[eval_schema][metric] += tmp_results[eval_schema][metric]

            # Calculate global precision and recall

            self.results = compute_precision_recall_wrapper(self.results)

            # Aggregate results by entity type

            for e_type in self.tags:

                for eval_schema in tmp_agg_results[e_type]:

                    for metric in tmp_agg_results[e_type][eval_schema]:

                        self.evaluation_agg_entities_type[e_type][eval_schema][metric] += tmp_agg_results[e_type][eval_schema][metric]

                # Calculate precision recall at the individual entity level

                self.evaluation_agg_entities_type[e_type] = compute_precision_recall_wrapper(self.evaluation_agg_entities_type[e_type])

        return self.results, self.evaluation_agg_entities_type


def collect_named_entities(tokens):
    """
    Creates a list of Entity named-tuples, storing the entity type and the start and end
    offsets of the entity.

    :param tokens: a list of tags
    :return: a list of Entity named-tuples
    """

    named_entities = []
    start_offset = None
    end_offset = None
    ent_type = None

    for offset, token_tag in enumerate(tokens):

        if token_tag == 'O':
            if ent_type is not None and start_offset is not None:
                end_offset = offset - 1
                named_entities.append(Entity(ent_type, start_offset, end_offset))
                start_offset = None
                end_offset = None
                ent_type = None

        elif ent_type is None:
            ent_type = token_tag[2:]
            start_offset = offset

        elif ent_type != token_tag[2:] or (ent_type == token_tag[2:] and token_tag[:1] == 'B'):

            end_offset = offset - 1
            named_entities.append(Entity(ent_type, start_offset, end_offset))

            # start of a new entity
            ent_type = token_tag[2:]
            start_offset = offset
            end_offset = None

    # catches an entity that goes up until the last token

    if ent_type is not None and start_offset is not None and end_offset is None:
        named_entities.append(Entity(ent_type, start_offset, len(tokens)-1))

    return named_entities


def compute_metrics(true_named_entities, pred_named_entities, tags):


    eval_metrics = {'correct': 0, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'precision': 0, 'recall': 0}

    # overall results
    
    evaluation = {
        'strict': deepcopy(eval_metrics),
        'ent_type': deepcopy(eval_metrics),
        'partial': deepcopy(eval_metrics),
        'exact': deepcopy(eval_metrics)
    }

    # results by entity type

    evaluation_agg_entities_type = {e: deepcopy(evaluation) for e in tags}

    # keep track of entities that overlapped

    true_which_overlapped_with_pred = []

    # Subset into only the tags that we are interested in.
    # NOTE: we remove the tags we don't want from both the predicted and the
    # true entities. This covers the two cases where mismatches can occur:
    #
    # 1) Where the model predicts a tag that is not present in the true data
    # 2) Where there is a tag in the true data that the model is not capable of
    # predicting.

    true_named_entities = [ent for ent in true_named_entities if ent.e_type in tags]
    pred_named_entities = [ent for ent in pred_named_entities if ent.e_type in tags]

    # go through each predicted named-entity

    for pred in pred_named_entities:
        found_overlap = False

        # Check each of the potential scenarios in turn. See
        # http://www.davidsbatista.net/blog/2018/05/09/Named_Entity_Evaluation/
        # for scenario explanation.

        # Scenario I: Exact match between true and pred

        if pred in true_named_entities:
            true_which_overlapped_with_pred.append(pred)
            evaluation['strict']['correct'] += 1
            evaluation['ent_type']['correct'] += 1
            evaluation['exact']['correct'] += 1
            evaluation['partial']['correct'] += 1

            # for the agg. by e_type results
            evaluation_agg_entities_type[pred.e_type]['strict']['correct'] += 1
            evaluation_agg_entities_type[pred.e_type]['ent_type']['correct'] += 1
            evaluation_agg_entities_type[pred.e_type]['exact']['correct'] += 1
            evaluation_agg_entities_type[pred.e_type]['partial']['correct'] += 1

        else:

            # check for overlaps with any of the true entities

            for true in true_named_entities:

                pred_range = range(pred.start_offset, pred.end_offset)
                true_range = range(true.start_offset, true.end_offset)

                # Scenario IV: Offsets match, but entity type is wrong

                if true.start_offset == pred.start_offset and pred.end_offset == true.end_offset \
                        and true.e_type != pred.e_type:

                    # overall results
                    evaluation['strict']['incorrect'] += 1
                    evaluation['ent_type']['incorrect'] += 1
                    evaluation['partial']['correct'] += 1
                    evaluation['exact']['correct'] += 1

                    # aggregated by entity type results
                    evaluation_agg_entities_type[true.e_type]['strict']['incorrect'] += 1
                    evaluation_agg_entities_type[true.e_type]['ent_type']['incorrect'] += 1
                    evaluation_agg_entities_type[true.e_type]['partial']['correct'] += 1
                    evaluation_agg_entities_type[true.e_type]['exact']['correct'] += 1

                    true_which_overlapped_with_pred.append(true)
                    found_overlap = True

                    break

                # check for an overlap i.e. not exact boundary match, with true entities

                elif find_overlap(true_range, pred_range):

                    true_which_overlapped_with_pred.append(true)

                    # Scenario V: There is an overlap (but offsets do not match
                    # exactly), and the entity type is the same.
                    # 2.1 overlaps with the same entity type

                    if pred.e_type == true.e_type:

                        # overall results
                        evaluation['strict']['incorrect'] += 1
                        evaluation['ent_type']['correct'] += 1
                        evaluation['partial']['partial'] += 1
                        evaluation['exact']['incorrect'] += 1

                        # aggregated by entity type results
                        evaluation_agg_entities_type[true.e_type]['strict']['incorrect'] += 1
                        evaluation_agg_entities_type[true.e_type]['ent_type']['correct'] += 1
                        evaluation_agg_entities_type[true.e_type]['partial']['partial'] += 1
                        evaluation_agg_entities_type[true.e_type]['exact']['incorrect'] += 1

                        found_overlap = True

                        break

                    # Scenario VI: Entities overlap, but the entity type is
                    # different.

                    else:
                        # overall results
                        evaluation['strict']['incorrect'] += 1
                        evaluation['ent_type']['incorrect'] += 1
                        evaluation['partial']['partial'] += 1
                        evaluation['exact']['incorrect'] += 1

                        # aggregated by entity type results
                        # Results against the true entity

                        evaluation_agg_entities_type[true.e_type]['strict']['incorrect'] += 1
                        evaluation_agg_entities_type[true.e_type]['partial']['partial'] += 1
                        evaluation_agg_entities_type[true.e_type]['ent_type']['incorrect'] += 1
                        evaluation_agg_entities_type[true.e_type]['exact']['incorrect'] += 1

                        # Results against the predicted entity

                        # evaluation_agg_entities_type[pred.e_type]['strict']['spurious'] += 1

                        found_overlap = True

                        break

            # Scenario II: Entities are spurious (i.e., over-generated).

            if not found_overlap:

                # Overall results

                evaluation['strict']['spurious'] += 1
                evaluation['ent_type']['spurious'] += 1
                evaluation['partial']['spurious'] += 1
                evaluation['exact']['spurious'] += 1

                # Aggregated by entity type results

                # NOTE: when pred.e_type is not found in tags
                # or when it simply does not appear in the test set, then it is
                # spurious, but it is not clear where to assign it at the tag
                # level. In this case, it is applied to all target_tags
                # found in this example. This will mean that the sum of the
                # evaluation_agg_entities will not equal evaluation.

                for true in tags:                    

                    evaluation_agg_entities_type[true]['strict']['spurious'] += 1
                    evaluation_agg_entities_type[true]['ent_type']['spurious'] += 1
                    evaluation_agg_entities_type[true]['partial']['spurious'] += 1
                    evaluation_agg_entities_type[true]['exact']['spurious'] += 1

    # Scenario III: Entity was missed entirely.

    for true in true_named_entities:
        if true in true_which_overlapped_with_pred:
            continue
        else:
            # overall results
            evaluation['strict']['missed'] += 1
            evaluation['ent_type']['missed'] += 1
            evaluation['partial']['missed'] += 1
            evaluation['exact']['missed'] += 1

            # for the agg. by e_type
            evaluation_agg_entities_type[true.e_type]['strict']['missed'] += 1
            evaluation_agg_entities_type[true.e_type]['ent_type']['missed'] += 1
            evaluation_agg_entities_type[true.e_type]['partial']['missed'] += 1
            evaluation_agg_entities_type[true.e_type]['exact']['missed'] += 1

    # Compute 'possible', 'actual' according to SemEval-2013 Task 9.1 on the
    # overall results, and use these to calculate precision and recall.

    for eval_type in evaluation:
        evaluation[eval_type] = compute_actual_possible(evaluation[eval_type])

    # Compute 'possible', 'actual', and precision and recall on entity level
    # results. Start by cycling through the accumulated results.

    for entity_type, entity_level in evaluation_agg_entities_type.items():

        # Cycle through the evaluation types for each dict containing entity
        # level results.

        for eval_type in entity_level:

            evaluation_agg_entities_type[entity_type][eval_type] = compute_actual_possible(
                entity_level[eval_type]
            )

    return evaluation, evaluation_agg_entities_type


def find_overlap(true_range, pred_range):
    """Find the overlap between two ranges

    Find the overlap between two ranges. Return the overlapping values if
    present, else return an empty set().

    Examples:

    >>> find_overlap((1, 2), (2, 3))
    2
    >>> find_overlap((1, 2), (3, 4))
    set()
    """

    true_set = set(true_range)
    pred_set = set(pred_range)

    overlaps = true_set.intersection(pred_set)

    return overlaps


def compute_actual_possible(results):
    """
    Takes a result dict that has been output by compute metrics.
    Returns the results dict with actual, possible populated.

    When the results dicts is from partial or ent_type metrics, then
    partial_or_type=True to ensure the right calculation is used for
    calculating precision and recall.
    """

    correct = results['correct']
    incorrect = results['incorrect']
    partial = results['partial']
    missed = results['missed']
    spurious = results['spurious']

    # Possible: number annotations in the gold-standard which contribute to the
    # final score

    possible = correct + incorrect + partial + missed

    # Actual: number of annotations produced by the NER system

    actual = correct + incorrect + partial + spurious

    results["actual"] = actual
    results["possible"] = possible

    return results


def compute_precision_recall(results, partial_or_type=False):
    """
    Takes a result dict that has been output by compute metrics.
    Returns the results dict with precison and recall populated.

    When the results dicts is from partial or ent_type metrics, then
    partial_or_type=True to ensure the right calculation is used for
    calculating precision and recall.
    """

    actual = results["actual"]
    possible = results["possible"]
    partial = results['partial']
    correct = results['correct']

    if partial_or_type:
        precision = (correct + 0.5 * partial) / actual if actual > 0 else 0
        recall = (correct + 0.5 * partial) / possible if possible > 0 else 0

    else:
        precision = correct / actual if actual > 0 else 0
        recall = correct / possible if possible > 0 else 0

    results["precision"] = precision
    results["recall"] = recall

    return results


def compute_precision_recall_wrapper(results):
    """
    Wraps the compute_precision_recall function and runs on a dict of results
    """

    results_a = {key: compute_precision_recall(value, True) for key, value in results.items() if
                 key in ['partial', 'ent_type']}
    results_b = {key: compute_precision_recall(value) for key, value in results.items() if
                 key in ['strict', 'exact']}

    results = {**results_a, **results_b}

    return results


In [10]:
def map_index2label(text, labels):
  index2label = {}
  start_index = 0
  for i, word in enumerate(text):
    end_index = start_index + len(word)
    index2label[(start_index, end_index)] = labels[i]
    start_index = end_index + 1
  return index2label

In [17]:
def benchmark(texts, labels_list, model, defined_labels_to_evaluate, tag_map):
  y_true = []
  y_pred = []
  index = 0
  for text, labels in zip(texts, labels_list):
    index2label_true = map_index2label(text, labels)
    y_true += [value for key, value in index2label_true.items()]
    model_result = model(" ".join(text))
    index2label_pred = {}
    for key, value in index2label_true.items():
      temp = []
      for item in model_result:
        if item['start'] >= key[0] and item['end'] <= key[1]:
          temp.append(item['entity'])
      index2label_pred[(key[0], key[1])] = temp[0]
    y_pred += [ tag_map[value] for key, value in index2label_pred.items()]
    index += 1
    if index%10 == 0:
      print(index)
  evaluator = Evaluator([y_true], [y_pred], tags=defined_labels_to_evaluate)
  return evaluator.evaluate()

### Albert

In [12]:
tokenizer = AutoTokenizer.from_pretrained("m3hrdadfi/albert-fa-base-v2-ner")
model = AutoModelForTokenClassification.from_pretrained("m3hrdadfi/albert-fa-base-v2-ner")
model.eval()
albert_ner = pipeline('ner', model=model, tokenizer=tokenizer, ignore_labels=[])

2021-02-05 11:46:18 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2021-02-05 11:46:18 urllib3.connectionpool DEBUG: https://huggingface.co:443 "HEAD /m3hrdadfi/albert-fa-base-v2-ner/resolve/main/config.json HTTP/1.1" 200 0
2021-02-05 11:46:18 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2021-02-05 11:46:18 urllib3.connectionpool DEBUG: https://huggingface.co:443 "HEAD /m3hrdadfi/albert-fa-base-v2-ner/resolve/main/spiece.model HTTP/1.1" 200 0
2021-02-05 11:46:18 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2021-02-05 11:46:19 urllib3.connectionpool DEBUG: https://huggingface.co:443 "HEAD /m3hrdadfi/albert-fa-base-v2-ner/resolve/main/tokenizer.json HTTP/1.1" 404 0
2021-02-05 11:46:19 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2021-02-05 11:46:19 urllib3.connectionpool DEBUG: https://huggingface.co:443 "HEAD /m3hrdadfi/albert-fa-base-

In [13]:
  tag_map = {
    "B-date": 'O',
    "B-event": "B-EVE",
    "B-facility": "B-ORG",
    "B-location": "B-LOC",
    "B-money": "O",
    "B-organization": "B-ORG",
    "B-person": "B-PER",
    "B-product": "O",
    "B-time": "O",
    "B-percent": "O",
    "I-date": "O",
    "I-event": "I-EVE",
    "I-facility": "I-ORG",
    "I-location": "I-LOC",
    "I-money": "O",
    "I-organization": "I-ORG",
    "I-person": "I-PER",
    "I-product": "O",
    "I-time": "O",
    "I-percent": "O",
    'O': 'O'
  }
benchmark(texts, tags, albert_ner, ['PER', 'LOC', 'ORG', 'EVE'], tag_map)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
2021-02-05 11:47:29 root INFO: Imported 1 predictions for 1 true examples


({'ent_type': {'actual': 601,
   'correct': 421,
   'incorrect': 41,
   'missed': 146,
   'partial': 0,
   'possible': 608,
   'precision': 0.7004991680532446,
   'recall': 0.6924342105263158,
   'spurious': 139},
  'exact': {'actual': 601,
   'correct': 442,
   'incorrect': 20,
   'missed': 146,
   'partial': 0,
   'possible': 608,
   'precision': 0.7354409317803661,
   'recall': 0.7269736842105263,
   'spurious': 139},
  'partial': {'actual': 601,
   'correct': 442,
   'incorrect': 0,
   'missed': 146,
   'partial': 20,
   'possible': 608,
   'precision': 0.7520798668885191,
   'recall': 0.743421052631579,
   'spurious': 139},
  'strict': {'actual': 601,
   'correct': 406,
   'incorrect': 56,
   'missed': 146,
   'partial': 0,
   'possible': 608,
   'precision': 0.6755407653910149,
   'recall': 0.6677631578947368,
   'spurious': 139}},
 {'EVE': {'ent_type': {'actual': 140,
    'correct': 1,
    'incorrect': 0,
    'missed': 13,
    'partial': 0,
    'possible': 14,
    'precision': 0

### Pars Bert

In [15]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased")
model = AutoModelForTokenClassification.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased")
model.eval()
parsbert_ner = pipeline('ner', model=model, tokenizer=tokenizer, ignore_labels=[])

2021-02-05 12:12:28 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2021-02-05 12:12:28 urllib3.connectionpool DEBUG: https://huggingface.co:443 "HEAD /HooshvareLab/bert-base-parsbert-ner-uncased/resolve/main/config.json HTTP/1.1" 200 0
2021-02-05 12:12:28 filelock DEBUG: Attempting to acquire lock 139791210457576 on /root/.cache/huggingface/transformers/cae79f4815c0cf33be75b6a1aa1d6236ab2646d25c5c26f5083ec2f31a6bc82f.c3f34538a8ec527cdbc5de919e808441d6737abb85fe85a68db484f12dceb83a.lock
2021-02-05 12:12:28 filelock INFO: Lock 139791210457576 acquired on /root/.cache/huggingface/transformers/cae79f4815c0cf33be75b6a1aa1d6236ab2646d25c5c26f5083ec2f31a6bc82f.c3f34538a8ec527cdbc5de919e808441d6737abb85fe85a68db484f12dceb83a.lock
2021-02-05 12:12:28 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2021-02-05 12:12:28 urllib3.connectionpool DEBUG: https://huggingface.co:443 "GET /HooshvareLab/bert-base-parsbert-ner-uncased/

2021-02-05 12:12:28 filelock DEBUG: Attempting to release lock 139791210457576 on /root/.cache/huggingface/transformers/cae79f4815c0cf33be75b6a1aa1d6236ab2646d25c5c26f5083ec2f31a6bc82f.c3f34538a8ec527cdbc5de919e808441d6737abb85fe85a68db484f12dceb83a.lock
2021-02-05 12:12:28 filelock INFO: Lock 139791210457576 released on /root/.cache/huggingface/transformers/cae79f4815c0cf33be75b6a1aa1d6236ab2646d25c5c26f5083ec2f31a6bc82f.c3f34538a8ec527cdbc5de919e808441d6737abb85fe85a68db484f12dceb83a.lock
2021-02-05 12:12:28 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443


2021-02-05 12:12:28 urllib3.connectionpool DEBUG: https://huggingface.co:443 "HEAD /HooshvareLab/bert-base-parsbert-ner-uncased/resolve/main/vocab.txt HTTP/1.1" 200 0
2021-02-05 12:12:28 filelock DEBUG: Attempting to acquire lock 139791210457576 on /root/.cache/huggingface/transformers/5df91a905c8036634a1cddbbcdab55e558aec2fd4309a294217d01f403e371f7.6699f2ee4745b6531f79b9781879071b6ace2d2768df83889391421fb44d4474.lock
2021-02-05 12:12:28 filelock INFO: Lock 139791210457576 acquired on /root/.cache/huggingface/transformers/5df91a905c8036634a1cddbbcdab55e558aec2fd4309a294217d01f403e371f7.6699f2ee4745b6531f79b9781879071b6ace2d2768df83889391421fb44d4474.lock
2021-02-05 12:12:28 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2021-02-05 12:12:29 urllib3.connectionpool DEBUG: https://huggingface.co:443 "GET /HooshvareLab/bert-base-parsbert-ner-uncased/resolve/main/vocab.txt HTTP/1.1" 200 1215509


2021-02-05 12:12:29 filelock DEBUG: Attempting to release lock 139791210457576 on /root/.cache/huggingface/transformers/5df91a905c8036634a1cddbbcdab55e558aec2fd4309a294217d01f403e371f7.6699f2ee4745b6531f79b9781879071b6ace2d2768df83889391421fb44d4474.lock
2021-02-05 12:12:29 filelock INFO: Lock 139791210457576 released on /root/.cache/huggingface/transformers/5df91a905c8036634a1cddbbcdab55e558aec2fd4309a294217d01f403e371f7.6699f2ee4745b6531f79b9781879071b6ace2d2768df83889391421fb44d4474.lock
2021-02-05 12:12:29 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443


2021-02-05 12:12:29 urllib3.connectionpool DEBUG: https://huggingface.co:443 "HEAD /HooshvareLab/bert-base-parsbert-ner-uncased/resolve/main/tokenizer.json HTTP/1.1" 404 0
2021-02-05 12:12:29 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2021-02-05 12:12:29 urllib3.connectionpool DEBUG: https://huggingface.co:443 "HEAD /HooshvareLab/bert-base-parsbert-ner-uncased/resolve/main/added_tokens.json HTTP/1.1" 404 0
2021-02-05 12:12:29 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2021-02-05 12:12:30 urllib3.connectionpool DEBUG: https://huggingface.co:443 "HEAD /HooshvareLab/bert-base-parsbert-ner-uncased/resolve/main/special_tokens_map.json HTTP/1.1" 200 0
2021-02-05 12:12:30 filelock DEBUG: Attempting to acquire lock 139790098628448 on /root/.cache/huggingface/transformers/9c399f248ec41d037b934b424658c33cd7050d5eb47eadfaaebef9859880f728.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock
2021-02-0

2021-02-05 12:12:30 filelock DEBUG: Attempting to release lock 139790098628448 on /root/.cache/huggingface/transformers/9c399f248ec41d037b934b424658c33cd7050d5eb47eadfaaebef9859880f728.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock
2021-02-05 12:12:30 filelock INFO: Lock 139790098628448 released on /root/.cache/huggingface/transformers/9c399f248ec41d037b934b424658c33cd7050d5eb47eadfaaebef9859880f728.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock
2021-02-05 12:12:30 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443


2021-02-05 12:12:30 urllib3.connectionpool DEBUG: https://huggingface.co:443 "HEAD /HooshvareLab/bert-base-parsbert-ner-uncased/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2021-02-05 12:12:30 filelock DEBUG: Attempting to acquire lock 139790069090400 on /root/.cache/huggingface/transformers/d9c378c1d878d4339ca3cfb8759681d66bb8a5b79d66e9905733939c2e47416e.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b.lock
2021-02-05 12:12:30 filelock INFO: Lock 139790069090400 acquired on /root/.cache/huggingface/transformers/d9c378c1d878d4339ca3cfb8759681d66bb8a5b79d66e9905733939c2e47416e.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b.lock
2021-02-05 12:12:30 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2021-02-05 12:12:30 urllib3.connectionpool DEBUG: https://huggingface.co:443 "GET /HooshvareLab/bert-base-parsbert-ner-uncased/resolve/main/tokenizer_config.json HTTP/1.1" 200 2


2021-02-05 12:12:30 filelock DEBUG: Attempting to release lock 139790069090400 on /root/.cache/huggingface/transformers/d9c378c1d878d4339ca3cfb8759681d66bb8a5b79d66e9905733939c2e47416e.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b.lock
2021-02-05 12:12:30 filelock INFO: Lock 139790069090400 released on /root/.cache/huggingface/transformers/d9c378c1d878d4339ca3cfb8759681d66bb8a5b79d66e9905733939c2e47416e.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b.lock


2021-02-05 12:12:31 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2021-02-05 12:12:31 urllib3.connectionpool DEBUG: https://huggingface.co:443 "HEAD /HooshvareLab/bert-base-parsbert-ner-uncased/resolve/main/config.json HTTP/1.1" 200 0
2021-02-05 12:12:31 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2021-02-05 12:12:31 urllib3.connectionpool DEBUG: https://huggingface.co:443 "HEAD /HooshvareLab/bert-base-parsbert-ner-uncased/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
2021-02-05 12:12:31 filelock DEBUG: Attempting to acquire lock 139790085635264 on /root/.cache/huggingface/transformers/74434d7ca32de74f5c4e1ef13ddcee2b2fe7b4a2861e8141267ef695011b0df7.aee1184fa3234eec730b51786e2e50883bb3b2e1312308fcb0999d13cb2ba8c5.lock
2021-02-05 12:12:31 filelock INFO: Lock 139790085635264 acquired on /root/.cache/huggingface/transformers/74434d7ca32de74f5c4e1ef13ddcee2b2fe7b4a2861e8141267ef695011b0df7.aee1184fa3234eec730b51

2021-02-05 12:12:48 filelock DEBUG: Attempting to release lock 139790085635264 on /root/.cache/huggingface/transformers/74434d7ca32de74f5c4e1ef13ddcee2b2fe7b4a2861e8141267ef695011b0df7.aee1184fa3234eec730b51786e2e50883bb3b2e1312308fcb0999d13cb2ba8c5.lock
2021-02-05 12:12:48 filelock INFO: Lock 139790085635264 released on /root/.cache/huggingface/transformers/74434d7ca32de74f5c4e1ef13ddcee2b2fe7b4a2861e8141267ef695011b0df7.aee1184fa3234eec730b51786e2e50883bb3b2e1312308fcb0999d13cb2ba8c5.lock


In [18]:
  tag_map = {
    "B-date": 'O',
    "B-event": "B-EVE",
    "B-facility": "B-ORG",
    "B-location": "B-LOC",
    "B-money": "O",
    "B-organization": "B-ORG",
    "B-person": "B-PER",
    "B-product": "O",
    "B-time": "O",
    "B-percent": "O",
    "I-date": "O",
    "I-event": "I-EVE",
    "I-facility": "I-ORG",
    "I-location": "I-LOC",
    "I-money": "O",
    "I-organization": "I-ORG",
    "I-person": "I-PER",
    "I-product": "O",
    "I-time": "O",
    "I-percent": "O",
    'O': 'O'
  }
benchmark(texts, tags, parsbert_ner, ['PER', 'LOC', 'ORG', 'EVE'], tag_map)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300


2021-02-05 12:18:45 root INFO: Imported 1 predictions for 1 true examples


({'ent_type': {'actual': 598,
   'correct': 434,
   'incorrect': 37,
   'missed': 137,
   'partial': 0,
   'possible': 608,
   'precision': 0.725752508361204,
   'recall': 0.7138157894736842,
   'spurious': 127},
  'exact': {'actual': 598,
   'correct': 452,
   'incorrect': 19,
   'missed': 137,
   'partial': 0,
   'possible': 608,
   'precision': 0.7558528428093646,
   'recall': 0.743421052631579,
   'spurious': 127},
  'partial': {'actual': 598,
   'correct': 452,
   'incorrect': 0,
   'missed': 137,
   'partial': 19,
   'possible': 608,
   'precision': 0.7717391304347826,
   'recall': 0.759046052631579,
   'spurious': 127},
  'strict': {'actual': 598,
   'correct': 419,
   'incorrect': 52,
   'missed': 137,
   'partial': 0,
   'possible': 608,
   'precision': 0.7006688963210702,
   'recall': 0.6891447368421053,
   'spurious': 127}},
 {'EVE': {'ent_type': {'actual': 128,
    'correct': 1,
    'incorrect': 0,
    'missed': 13,
    'partial': 0,
    'possible': 14,
    'precision': 0.0

### XLMR

In [22]:
nlp_ner = pipeline(
    "ner",
    model="jplu/tf-xlm-r-ner-40-lang",
    tokenizer=(
        'jplu/tf-xlm-r-ner-40-lang',  
        {"use_fast": True}),
    framework="tf",
    ignore_labels=[],
)

2021-02-05 12:32:27 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2021-02-05 12:32:27 urllib3.connectionpool DEBUG: https://huggingface.co:443 "HEAD /jplu/tf-xlm-r-ner-40-lang/resolve/main/config.json HTTP/1.1" 200 0
2021-02-05 12:32:27 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2021-02-05 12:32:27 urllib3.connectionpool DEBUG: https://huggingface.co:443 "HEAD /jplu/tf-xlm-r-ner-40-lang/resolve/main/sentencepiece.bpe.model HTTP/1.1" 200 0
2021-02-05 12:32:27 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2021-02-05 12:32:28 urllib3.connectionpool DEBUG: https://huggingface.co:443 "HEAD /jplu/tf-xlm-r-ner-40-lang/resolve/main/tokenizer.json HTTP/1.1" 404 0
2021-02-05 12:32:28 urllib3.connectionpool DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2021-02-05 12:32:28 urllib3.connectionpool DEBUG: https://huggingface.co:443 "HEAD /jplu/tf-xlm-r-ner-40-lang/resolv

In [23]:
  tag_map = {
    "PER": 'B-PER',
    "LOC": "B-LOC",
    "ORG": "B-ORG",
    'O': 'O'
  }
benchmark(texts, tags, nlp_ner, ['PER', 'LOC', 'ORG'], tag_map)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300


2021-02-05 12:34:17 root INFO: Imported 1 predictions for 1 true examples


({'ent_type': {'actual': 1223,
   'correct': 247,
   'incorrect': 49,
   'missed': 298,
   'partial': 0,
   'possible': 594,
   'precision': 0.2019623875715454,
   'recall': 0.4158249158249158,
   'spurious': 927},
  'exact': {'actual': 1223,
   'correct': 296,
   'incorrect': 0,
   'missed': 298,
   'partial': 0,
   'possible': 594,
   'precision': 0.24202780049059688,
   'recall': 0.4983164983164983,
   'spurious': 927},
  'partial': {'actual': 1223,
   'correct': 296,
   'incorrect': 0,
   'missed': 298,
   'partial': 0,
   'possible': 594,
   'precision': 0.24202780049059688,
   'recall': 0.4983164983164983,
   'spurious': 927},
  'strict': {'actual': 1223,
   'correct': 247,
   'incorrect': 49,
   'missed': 298,
   'partial': 0,
   'possible': 594,
   'precision': 0.2019623875715454,
   'recall': 0.4158249158249158,
   'spurious': 927}},
 {'LOC': {'ent_type': {'actual': 1092,
    'correct': 131,
    'incorrect': 34,
    'missed': 56,
    'partial': 0,
    'possible': 221,
    'pre